In [1]:
# FILENAMES real recording in sparcs
import numpy as np
from class_and_functions_for_combinations import *

names = [
    # 'CW50_0_0',
    # 'CW50_0_5',
    # 'CW50_0_10',
    # 'CW50_0_15',
    # 'CW50_30_0',
    # 'CW50_30_5',
    # 'CW50_30_10',
    # 'CW50_30_15',
    # 'ACW50_0_0',
    # 'ACW50_0_5',
    # 'ACW50_0_10',
    # 'ACW50_0_15',
    # 'ACW50_30_0',
    # 'ACW50_30_5',
    # 'ACW50_30_10',
    # 'ACW50_30_15',

    'cw0',
    'cw2',
    'cw4',
    'cw6',
    'cw8',
    'cw10',
    'cw12',
    'cw14',
    'acw0',
    'acw2',
    'acw4',
    'acw6',
    'acw8',
    'acw10',
    'acw12',
    'acw14',
    'train18'

    # 'sim_ds_0',
    # 'sim_ds_5',
    # 'sim_ds_10',
    # 'sim_ds_15',
    # 'sim_ds_20',
    # 'sim_ds_25',
    # 'sim_ds_30',   
    #  
    # 'sim_ds_0',
    # 'sim_ds_2',
    # 'sim_ds_4',
    # 'sim_ds_6',
    # 'sim_ds_8',
    # 'sim_ds_10',
    # 'sim_ds_12',
    # 'sim_ds_14',
    # 'sim_ds_16',
    # 'sim_ds_18',
    # 'sim_ds_20',
]

#NOTES: 
# -CW50_0_0 is only one lap

CAM_Z = 0.195
CAM_ROLL = np.deg2rad(-2)
CAM_PITCH = np.deg2rad(24)

VICON_STATIC_FRAMES_DELAY = 5 #5
YAW_ERROR = np.deg2rad(2.5)



/home/irong/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
assert False, 'Did u do a run all, didnt u? It\'s fine but ur journey ends here'

In [ ]:
# create clean datasetes: compensate vicon delay + associate more precise locs
#compensate vicon delay function
import numpy as np
def make_yaws_incremental(samples):
    assert samples.shape[1] == 3, 'sample must be 3d, x,y,yaw'
    yaws = samples[:,2]
    assert np.all(yaws >= -np.pi) and np.all(yaws <= np.pi), 'yaws must be in [-pi, pi]'
    increments = np.zeros_like(yaws)
    for i in range(1, len(yaws)):
        if yaws[i] < np.deg2rad(-175) and yaws[i-1] > np.deg2rad(175):
            increments[i:] += 2*np.pi
        elif yaws[i] > np.deg2rad(175) and yaws[i-1] < np.deg2rad(-175):
            increments[i:] -= 2*np.pi
    yaws += increments
    samples[:,2] = yaws
    return samples

def compensate_vicon_delay(imgs, locs):
    """
    Compensate for the delay between the Vicon and the images.
    """
    locs = locs[:-VICON_STATIC_FRAMES_DELAY]
    imgs = imgs[VICON_STATIC_FRAMES_DELAY:]
    assert len(imgs) == len(locs)
    
    xs, ys, yaws = locs[:,0], locs[:,1], locs[:,2]
    yaws = yaws - YAW_ERROR
    locs = np.stack([xs, ys, yaws], axis=1)

    return imgs, locs

def resample(target, full_array, factor):
    assert target * factor <= len(full_array), f'target is too long, we dont have enough sample from the full array, target: {target}, full_array: {len(full_array)}, factor: {factor}'
    assert locs.shape[1] == 3, 'locs must be 3d, x,y,yaw'
    L = target
    resampled = np.zeros((target,3))
    samples = full_array[-int(round(L * factor)):]

    for i in range(3):
        resampled[:,i] = np.interp(np.linspace(0, len(samples), L), np.arange(len(samples)), samples[:,i])
    return resampled

def smooth_samples(sample):
    from scipy.signal import savgol_filter
    assert sample.shape[1] == 3, 'sample must be 3d, x,y,yaw'
    for i in range(3):
        sample[:,i] = savgol_filter(sample[:,i], 20, 5)
    return sample

import matplotlib.pyplot as plt
IMG_FOLDER = 'saved_tests/new_tests/'
LOCS_FOLDER = 'saved_tests/new_tests_locs/l'  
SAVE_FODLER = 'saved_tests/'

for name in names:
    print(f'processing: {name}')
    log = np.load(IMG_FOLDER + name + '.npz')
    imgs, locs = log['imgs'],log['locs']
    acc_locs_log = np.load(LOCS_FOLDER + name + '.npz')
    acc_locs = acc_locs_log['locs']
    assert len(imgs) == len(locs), f'imgs and locs have different length: {len(imgs)} != {len(locs)}'
    l = len(imgs)

    #make yaws incremental
    locs = make_yaws_incremental(locs)
    acc_locs = make_yaws_incremental(acc_locs)

    resample_factor = 1.0 
    max_resample_factor = min(len(acc_locs) / l * 1.0, 1.02)
    factors = np.linspace(1.0, max_resample_factor, 10000)
    min_diff = np.inf
    for factor in factors:
        new_locs = resample(l, acc_locs, factor)
        diff = np.mean(np.linalg.norm(locs[:,:2] - new_locs[:,:2], axis=1))
        if diff < min_diff:
            min_diff = diff
            resample_factor = factor
    
    print(f'resample_factor: {resample_factor}')
    new_locs = resample(l, acc_locs, resample_factor)

    #compensate vicon delay
    tmp_imgs = imgs.copy()
    imgs, new_locs_comp = compensate_vicon_delay(tmp_imgs, new_locs)
    imgs, locs_comp = compensate_vicon_delay(tmp_imgs, locs)
    new_locs_comp = smooth_samples(new_locs_comp)
    locs_comp = smooth_samples(locs_comp)

    np.savez_compressed(SAVE_FODLER + name + '.npz', imgs=imgs, locs=new_locs_comp)
    
    print(f'locs shape: {locs.shape}\nnew_locs shape: {new_locs.shape}\nimgs shape: {imgs.shape}')
    print(f'locs_comp shape: {locs_comp.shape}\nnew_locs_comp shape: {new_locs_comp.shape}\nimgs shape: {imgs.shape}')

    diff = np.linalg.norm(locs[:,:2] - new_locs[:,:2], axis=1)

    #plot
    N = 500
    L = locs_comp.shape[0]
    idxs = np.arange(0,1000)

    plocs = locs[idxs]
    pnew_locs = new_locs[idxs]
    plocs_comp = locs_comp[idxs]
    pnew_locs_comp = new_locs_comp[idxs]
    pdiff = diff[idxs]

    plt.figure(figsize=(20,15))
    plt.subplot(3,1,1)
    plt.plot(pdiff, label='diff')
    plt.plot(plocs[:,0], 'r--', label='locs')
    plt.plot(pnew_locs[:,0], 'b--', label='new_locs')
    plt.plot(plocs_comp[:,0], 'r-', label='locs_comp')
    plt.plot(pnew_locs_comp[:,0], 'b-', label='new_locs_comp')
    plt.legend()
    plt.grid()

    plt.subplot(3,1,2)
    # plt.plot(pdiff, label='diff')
    plt.plot(plocs[:,1], 'r--', label='locs')
    plt.plot(pnew_locs[:,1], 'b--', label='new_locs')
    plt.plot(plocs_comp[:,1], 'r-', label='locs_comp')
    plt.plot(pnew_locs_comp[:,1], 'b-', label='new_locs_comp')
    plt.legend()
    plt.grid()

    plt.subplot(3,1,3)
    # plt.plot(pdiff, label='diff')
    plt.plot(plocs[:,2], 'r--', label='yaws')
    plt.plot(pnew_locs[:,2], 'b--', label='new_yaws')
    plt.plot(plocs[:,2]-pnew_locs[:,2], 'r-', label='yaws_diff')
    plt.plot(plocs_comp[:,2]-pnew_locs_comp[:,2], 'b-', label='yaws_comp_diff')
    # plt.plot(plocs_comp[:,2], 'r-', label='locs_comp')
    # plt.plot(pnew_locs_comp[:,2], 'b-', label='new_locs_comp')
    plt.legend()
    plt.grid()

    plt.show()



In [ ]:
# generating simulator version of real video
from unittest.mock import CallableMixin
import numpy as np
import rospy 
import rospkg 
from gazebo_msgs.msg import ModelState 
from gazebo_msgs.srv import SetModelState
from cv_bridge import CvBridge
from sensor_msgs.msg import Image, Range
import os
from time import sleep
import cv2 as cv
from Simulator.src.helper_functions import *

path = np.load('sparcs/sparcs_path.npy').T
lane_int = np.load('sparcs/sparcs_path_int.npy').T
lane_ext = np.load('sparcs/sparcs_path_ext.npy').T


frame = None
bridge = CvBridge()

#create rosservice
rospy.init_node('gazebo_move')
#create subscriber for automobile/image_raw
def camera_callback(data) -> None:
    """Receive and store camera frame
    :acts on: self.frame
    """        
    global frame, bridge
    frame = bridge.imgmsg_to_cv2(data, "bgr8")

camera_sub = rospy.Subscriber('/automobile/image_raw', Image, camera_callback)

cv.namedWindow('frame', cv.WINDOW_NORMAL)
cv.resizeWindow('frame', 640, 480)
cv.namedWindow('real', cv.WINDOW_NORMAL)
cv.resizeWindow('real', 640, 480)

rospy.wait_for_service('/gazebo/set_model_state')

state_msg = ModelState()
state_msg.model_name = 'automobile'
state_msg.pose.position.x = 0
state_msg.pose.position.y = 0
state_msg.pose.position.z = 0.032939
state_msg.pose.orientation.x = 0
state_msg.pose.orientation.y = 0
state_msg.pose.orientation.z = 0
state_msg.pose.orientation.w = 0

please_break = False
for name in names:
    print(name)
    log = np.load('saved_tests/'+name+'.npz')
    imgs, locs = log['imgs'],log['locs']
    #get ony the first lap
    # imgs = imgs[:1473//2] #//2
    # locs = locs[:1473//2] #

    cv.resizeWindow('frame', 640, 480)
    cv.resizeWindow('real', 640, 480)

    sim_imgs = np.zeros_like(imgs)


    for i,(img,l) in enumerate(zip(imgs,locs)):
        x = l[0] + 2.5
        y = l[1] + 2.5 - 15.0
        yaw = l[2]
        x,y,yaw = rear2com(x,y,yaw) #convert from rear axle to center of mass

        qx = 0
        qy = 0
        qz = np.sin(yaw/2)
        qw = np.cos(yaw/2)

        state_msg.pose.position.x = x
        state_msg.pose.position.y = y
        state_msg.pose.orientation.z = qz
        state_msg.pose.orientation.w = qw

        set_state = rospy.ServiceProxy('/gazebo/set_model_state', SetModelState)
        resp = set_state(state_msg)
        sleep(0.008)
        #convert frame to gray
        tmp_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        tmp_frame = cv.resize(tmp_frame, (320,240), interpolation = cv.INTER_AREA)
        sim_imgs[i] = tmp_frame

        sim_img = cv.resize(frame, (320,240), interpolation = cv.INTER_AREA)

        x,y,yaw = l
        D = 5
        img, _ = project_onto_frame2(img,path[::D], x,y,yaw, color=(0,0,255), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
        img, _ = project_onto_frame2(img,lane_int[::D], x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
        img, _ = project_onto_frame2(img,lane_ext[::D], x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
        sim_img, _ = project_onto_frame2(sim_img,path[::D], x,y,yaw, color=(0,0,255), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
        sim_img, _ = project_onto_frame2(sim_img,lane_int[::D], x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
        sim_img, _ = project_onto_frame2(sim_img,lane_ext[::D], x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)



        cv.imshow('frame', sim_img)
        cv.imshow('real', img)
        #check if its ESC
        please_break = cv.waitKey(1) == 27
        
        if please_break:
            break
        # os.system('rosservice call /gazebo/set_model_state "{model_state: { model_name: '+"'automobile'"+', pose: { position: { x: '+str(x)+', y: '+str(y)+', z: 0.032939 }, orientation: { x: '+str(qx)+', y: '+str(qy)+', z: '+str(qz)+', w: '+str(qw)+' } }, twist: { linear: { x: 0.0, y: 0.0, z: 0.0 }, angular: { x: 0.0, y: 0.0, z: 0.0 } }, reference_frame: '+"'world'"+' } }"')

    if please_break:
        break
    else:
        np.savez_compressed('saved_tests/'+name + '_SIM.npz', imgs=sim_imgs, locs=locs)


cv.destroyAllWindows()


In [ ]:
#check videos and sim version + data plotting
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from Simulator.src.helper_functions import *
from path_nn_controller import PathPlanning, Controller, Detection
from time import time, sleep
DIST_AHEAD = 0.4

NN = Detection() #initialize neural network

lane_int = np.load('sparcs/sparcs_path_int.npy').T
lane_ext = np.load('sparcs/sparcs_path_ext.npy').T

onnx_model = cv.dnn.readNetFromONNX('tmp/models/')


please_break = False

for name in names:
    print(name)
    log = np.load('saved_tests/'+name+'.npz')
    imgs, locs = log['imgs'],log['locs']
    sim_log = np.load('saved_tests/'+name+'_SIM.npz')
    sim_imgs, sim_locs = sim_log['imgs'],sim_log['locs']
    #get ony the first lap
    # imgs = imgs[:1473//2] #//2
    # locs = locs[:1473//2] #
    path = np.load('sparcs/sparcs_path_precise.npy').T
    print(f'frames: {len(imgs)}, locs: {len(locs)}')
    print(f'frame dim {imgs[0].shape}, locs dim {locs[0].shape}')

    #check if name starts with CW or ACW
    if name[:3] == 'ACW' or name[:3] == 'acw':
        path = np.flip(path, axis = 0) #if reversed/anti-clockwise

    hes = np.zeros(imgs.shape[0]) #heading error
    est_hes = np.zeros(imgs.shape[0]) #estimated heading error
    est_hes_sim = np.zeros(imgs.shape[0]) #estimated heading error simulator
    dists = np.zeros(imgs.shape[0]) #distances to path

    #calculations over all frames
    for i, (img,sim_img, (x,y,yaw)) in enumerate(zip(imgs,sim_imgs, locs)):
        img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
        sim_img = cv.cvtColor(sim_img, cv.COLOR_GRAY2BGR)

        he, pa, d = get_heading_error(x,y,yaw,path, DIST_AHEAD)
        # sleep(10)
        # est_he, est_pa = NN.estimate_he(img, distance_point_ahead=DIST_AHEAD, show_ROI=False)
        # est_he_sim, est_pa_sim = NN.estimate_he(sim_img, distance_point_ahead=DIST_AHEAD, show_ROI=False)
        
        est_he = get_est_heading_error(img, onnx_model)
        est_he_sim = get_est_heading_error(sim_img, onnx_model)

        hes[i] = np.rad2deg(he)
        est_hes[i] = np.rad2deg(est_he)
        est_hes_sim[i] = np.rad2deg(est_he_sim)
        dists[i] = d

    ###############################################################
    # plotting, showing results

    #create matplotlib figure with 2 subplots
    fig, ax = plt.subplots(1,2, figsize=(25,10))
    fig.suptitle(name)
    ax[0].set_title('Path')
    ax[1].set_title('Heading errror')

    #plot path
    ax[0].plot(path[:,0], path[:,1], label='reference')
    ax[0].plot(locs[:,0], locs[:,1], label='actual')
    ax[0].set_aspect('equal')
    ax[0].legend()
    ax[0].set_xlabel('x [m]')
    ax[0].set_ylabel('y [m]')
    ax[0].set_xlim([1.5-3,1.5+3])
    ax[0].set_ylim([-.5,5.5])
    ax[0].grid()

    #plot heading errors
    #create twinx
    ax2 = ax[1].twinx()
    ax2.plot(dists, '-r', label='dist ref path')
    # ax2.legend()
    ax2.set_ylabel('distance [m]')
    ax2.set_ylim([-0.15,0.15])
    ax[1].plot(hes, label='true')
    ax[1].plot(est_hes_sim, label='estimated_sim')
    ax[1].plot(est_hes, label='estimated')
    ax[1].legend()
    ax[1].set_xlabel('frame')
    ax[1].set_ylabel('heading error [deg]')
    ax[1].set_ylim([-50,50])
    ax[1].grid()
    plt.show(block=False)

    # #show images
    # cv.namedWindow(name, cv.WINDOW_NORMAL)
    # cv.resizeWindow(name, 640, 480)
    # cv.namedWindow(name+'_SIM', cv.WINDOW_NORMAL)
    # cv.resizeWindow(name+'_SIM', 640, 480)
    # for img,sim_img, he, est_he, est_he_sim, (x,y,yaw) in zip(imgs,sim_imgs, hes, est_hes, est_hes_sim, locs):
    #     img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
    #     sim_img = cv.cvtColor(sim_img, cv.COLOR_GRAY2BGR)   
        
    #     he = np.deg2rad(he)
    #     est_he = np.deg2rad(est_he)
    #     est_he_sim = np.deg2rad(est_he_sim)

    #     img = draw_angle(img, he, color=(200-50, 200-50, 100-50))
    #     img = draw_angle(img, est_he, color=(200-50, 100-50, 200-50))
    #     sim_img = draw_angle(sim_img, he, color=(200-50, 200-50, 100-50))
    #     sim_img = draw_angle(sim_img, est_he_sim, color=(200-50, 100-50, 200-50))

    #     #decimate path
    #     points = np.array([path[::100,0], path[::100,1]]).T
    #     img, _ = project_onto_frame2(img,points, x,y,yaw, color=(0,0,255), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
    #     img, _ = project_onto_frame2(img,lane_int, x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
    #     img, _ = project_onto_frame2(img,lane_ext, x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
    #     sim_img, _ = project_onto_frame2(sim_img,points, x,y,yaw, color=(0,0,255), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
    #     sim_img, _ = project_onto_frame2(sim_img,lane_int, x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
    #     sim_img, _ = project_onto_frame2(sim_img,lane_ext, x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)



    #     cv.imshow(name, img)
    #     cv.imshow(name+'_SIM', sim_img)
    #     please_break = cv.waitKey(1) == 27
    #     if please_break:
    #         break
    #     # sleep(1/160.0)
    # if please_break:
    #     break
    # cv.destroyAllWindows()
    print(' ')

cv.destroyAllWindows()



In [ ]:
#check videos but NO sim version + data plotting
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from Simulator.src.helper_functions import *
from path_nn_controller import PathPlanning, Controller, Detection
from time import time, sleep
DIST_AHEAD = 0.5

MODEL_FOLDER = 'Simulator/models'
model_name = MODEL_FOLDER + '/lane_keeper.pt'
onnx_lane_keeper_path = MODEL_FOLDER + '/lane_keeper.onnx'

NN = Detection() #initialize neural network

lane_int = np.load('sparcs/sparcs_path_int.npy').T
lane_ext = np.load('sparcs/sparcs_path_ext.npy').T

onnx_file_name = 'tmp/models/tr_ds_sn18_he40_canny100_200_blur3_noise80_keep80_size32_length10000_archa_bs65536_lr3000_ep300_L1100_L210000_wd90_dr30.onnx'
onnx_file_name = onnx_lane_keeper_path


onnx_model = cv.dnn.readNetFromONNX(onnx_file_name)

please_break = False

for name in names:
    print(name)
    log = np.load('saved_tests/'+name+'.npz')
    imgs, locs = log['imgs'],log['locs']
    # imgs, locs = compensate_vicon_delay(imgs, locs)
    #get ony the first lap
    imgs = imgs[:1150] #//2
    locs = locs[:1150] #
    path = np.load('sparcs/sparcs_path_precise.npy').T
    print(f'frames: {len(imgs)}, locs: {len(locs)}')
    print(f'frame dim {imgs[0].shape}, locs dim {locs[0].shape}')

    #check if name starts with CW or ACW
    if name[:3] == 'ACW' or name[:3] == 'acw':
        path = np.flip(path, axis = 0) #if reversed/anti-clockwise

    

    hes = np.zeros(imgs.shape[0]) #heading error
    est_hes = np.zeros(imgs.shape[0]) #estimated heading error
    dists = np.zeros(imgs.shape[0]) #distances to path

    #calculations over all frames
    for i, (img, (x,y,yaw)) in enumerate(zip(imgs, locs)):
        img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)

        he, pa, d = get_heading_error(x,y,yaw,path, DIST_AHEAD)
        # sleep(10)
        # est_he, est_pa = NN.estimate_he(img, distance_point_ahead=DIST_AHEAD, show_ROI=False)
        est_he = get_est_heading_error(img, onnx_model)
        hes[i] = np.rad2deg(he)
        est_hes[i] = np.rad2deg(est_he)
        dists[i] = d

    ###############################################################
    # plotting, showing results

    #create matplotlib figure with 2 subplots
    fig, ax = plt.subplots(1,2, figsize=(25,10))
    fig.suptitle(name)
    ax[0].set_title('Path')
    ax[1].set_title('Heading errror')

    #plot path
    ax[0].plot(path[:,0], path[:,1], label='reference')
    ax[0].plot(locs[:,0], locs[:,1], label='actual')
    ax[0].set_aspect('equal')
    ax[0].legend()
    ax[0].set_xlabel('x [m]')
    ax[0].set_ylabel('y [m]')
    ax[0].set_xlim([1.5-3,1.5+3])
    ax[0].set_ylim([-.5,5.5])
    ax[0].grid()

    #plot heading errors
    ax[1].plot(hes, label='true')
    ax[1].plot(est_hes, label='estimated')
    ax[1].legend()
    ax[1].set_xlabel('frame')
    ax[1].set_ylabel('heading error [deg]')
    ax[1].set_ylim([-50,50])
    ax[1].grid()
    #create twinx
    ax2 = ax[1].twinx()
    ax2.plot(dists, '-r', label='dist ref path')
    # ax2.legend()
    ax2.set_ylabel('distance [m]')
    ax2.set_ylim([-0.15,0.15])
    plt.show(block=False)

    # #show images
    # cv.namedWindow(name, cv.WINDOW_NORMAL)
    # cv.resizeWindow(name, 640, 480)
    # for img, he, est_he,  (x,y,yaw) in zip(imgs, hes, est_hes, locs):
    #     img = cv.cvtColor(img, cv.COLOR_GRAY2BGR)
        
    #     he = np.deg2rad(he)
    #     est_he = np.deg2rad(est_he)

    #     img = draw_angle(img, he, color=(200-50, 200-50, 100-50))
    #     img = draw_angle(img, est_he, color=(200-50, 100-50, 200-50))

    #     #decimate path
    #     points = np.array([path[::100,0], path[::100,1]]).T
    #     img, _ = project_onto_frame2(img,points, x,y,yaw, color=(0,0,255), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
    #     img, _ = project_onto_frame2(img,lane_int, x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)
    #     img, _ = project_onto_frame2(img,lane_ext, x,y,yaw, color=(200,200,200), thickness=1, CAM_Z=0.2, CAM_FOV=1.085594795, CAM_PITCH=CAM_PITCH, CAM_ROLL=CAM_ROLL)

    #     cv.imshow(name, img)
    #     please_break = cv.waitKey(1) == 27
    #     if please_break:
    #         break
    #     # sleep(1/160.0)
    # if please_break:
    #     break
    # cv.destroyAllWindows()
    # print(' ')

cv.destroyAllWindows()

